# 2022/01/03/MON

> `평가 process에 대해 알아보자`

### - `정확도 ?` = 예측 결과가 동일한 데이터 건수 / 전체 예측 데이터 건수
    - 이진 부류의 경우 데이터 구성에 따라 ML 모델의 성능을 왜곡할 수 있기 때문에 정확도 수치 하나만 가지고 성능을 평가하는 건 위험함
    - 그 예를 살펴보자

In [3]:
from sklearn.base import BaseEstimator
import numpy as np

class MyDummyClassifier(BaseEstimator):
    # fit 메서드는 아무것도 학습하지 않음
    def fit(self, X, y=None):
        pass
    # predict() 메서드는 단순히 Sex feature 1이면 0 그렇지 않으면 1로 예측함
    def predict(self, X):
        pred = np.zeros((X.shape[0],1))
        for i in range(X.shape[0]) :
            if X['Sex'].iloc[i]==1 : 
                pred[i]=0
            else :
                pred[i]=1
        return pred

```python
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# 원본 데이터를 재로딩, 데이터 가공, 학습 데이터/테스트 데이터 분할
titanic_df= pd.read_csv('C:/Users/ehfus/Downloads/titanic/train.csv')
y_titanic_df=titanic_df['Survived']
X_titanic_df=titanic_df.drop('Survived',axis=1)
X_titanic_df=transform_features(X_titanic_df) # transform_features 함수 정의 안 했기 때문에 에러 발생(140p)
x_train,X_test,y_train,y_test = train_test_split(X_titanic_df, y_titanic_df,test_size=.2,random_state=0)

# 위에서 생성한 Dummy Classifier를 이용해 학습/예측/평가 수행
myclf=MydummyClassifier()
myclf.fit(X_train,y_train)

mypredictions=myclf.predict(X_train,y_train)

mypredictions = myclf.predict(X_test)
print('Dummy Classifier의 정확도는: {0:.4f}'.format(accuracy_score(y_test,mypredictions)))

```

Dummy Classifier의 정확도는 0.7877정도 나온다

즉 이렇게 단순한 알고리즘으로 예측을 하더라도 데이터의 구성에 따라 정확도의 결과는 약 78%정도로 높은 수치가 나올 수 있음. 따라서 정확도를 지표로 사용할 때는 매우 신중해야 함.

***특히 불균형한 레이블 값 분포에서 ML 모델의 성능을 판단할 경우 적합한 평가 지표가 아님, 예를 들어보자***

In [10]:
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd

class MyFakeClassifier(BaseEstimator):
    def fit(self, X, y):
        pass
    
    # 입력값으로 들어오는 X 데이터 세트의 크기만큼 모두 0값으로 만들어서 반환
    def predict(self, X):
        return np.zeros((len(X),1),dtype=bool)

# 사이킷런의 내장 데이터 세트인 load_digits()를 이용해 MNIST 데이터 로딩
digits = load_digits()

# digits 번호가 7번이면 True이고 이를 astype(int)로 1로 변환, 7번이 아니면 Fasle이고 0으로 변환.
y=(digits.target==7).astype(int)
X_train,X_test,y_train,y_test= train_test_split(digits.data,y,random_state=11)

# 불균형한 레이블 데이터 분포도 확인
print('레이블 테스트 세트 크기: ',y_test.shape)
print('테스트 세트 레이블 0과 1의 분포도')
print(pd.Series(y_test).value_counts())

# Dummy Classifier로 학습/예측/정확도 평가
fakeclf=MyFakeClassifier()
fakeclf.fit(X_train,y_train)
fakepred=fakeclf.predict(X_test)
print('모든 예측을 0으로 하여도 정확도는:{:.3f}'.format(accuracy_score(y_test,fakepred)))

레이블 테스트 세트 크기:  (450,)
테스트 세트 레이블 0과 1의 분포도
0    405
1     45
dtype: int64
모든 예측을 0으로 하여도 정확도는:0.900


- 이처럼 정확도 평가 지표는 불균형한 레이블 데이터 세트에서는 성능 수치로 사용 되어서는 안 된다. 여러가지 분류 지표와 함께 이용하자

### - `오차행렬 또는 혼동행렬 ?` 
    - 학습된 분류 모델이 예측을 수행하면서 얼마나 헷갈리고 있는지도 함께 보여주는 지표
    - TN,FP,FN,TP로 나뉘며, 앞문자는 예측값과 실제값이 '같은가/ 틀린가'를 의미, 뒤 문자는 예측 결과 값이 부정(0)/긍정(1)을 의미
    - 이 값을 조합해 정확도, 정밀도, 재현율 값을 알 수 있음

In [12]:
# 오차행렬 구하는 API
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, fakepred)

array([[405,   0],
       [ 45,   0]], dtype=int64)

- TN은 405개, FN은 45개이다.

- 정확도 = 예측 결과와 실제 값이 동일한 건수/전체 데이터 수 = (TN + TP) / (TN + FP + FN + TP)
- 정밀도 = TP / (FP + TP)
- 재현율 = TP = (FN + TP)

(158p 참고)

분류하려는 업무의 특성상 정밀도 또는 재현율이 특별히 강조되어야 할 경우 분류의 결정 임계값(Threshold)을 조정해 정밀도 또는 재현율의 수치를 높일 수 있다.

개별 데이터 별로 예측 확률을 반환하는 메서드 = predict_proba()
predict_proba() 메서드는 학습이 완료된 사이킷런 Classifier 객체에서 호출이 가능하며 테스트 feature 데이터 세트를 파라미터로 입력해주면 테스트 feature 레코드의 개별 클래스 예측 확률을 반환함. predict() 메서드와 유사하지만 단지 반환 결과가 예측 결과 클래스값이 아닌 예측 확률 결과이다.

(160p 참고)

- threshold 값을 조정해보자

In [13]:
from sklearn.preprocessing import Binarizer

X=[[1,-1,2],
  [2,0,0],
  [0,1.1,1.2]]

# X의 개별 원소들이 threshold 값보다 같거나 작으면 0을 크면 1을 반환
binarizer = Binarizer(threshold=1.1)
print(binarizer.fit_transform(X))

[[0. 0. 1.]
 [1. 0. 0.]
 [0. 0. 1.]]


이제 이 Binarizer를 이용해 사이킷런 predict()의 의사(pseudo)코드를 만들어보자

```python
from sklearn.preprocessing import Binarizer

# Binarizer의 threshold 설정값, 즉 분류 결정 임계값임.
custom_threshold=0.5

# predict_proba() 반환값의 두 번째 칼럼, 즉 Positive 클래스 칼럼 하나만 추출해 Binirizer를 적용
pred_proba_1=pred_proba[:,1].reshape(-1,1)

binarizer = Binarizer(threshold=cistom_threshold).fit(pred_proba_1)
custom_predict = binarizer.transform(pred_proba_1)

get_clf_eval(y_test, custom_predict)
```

이때 threshold 설정값을 0.4로 설정, 즉 분류 결정 임계값을 낮추면 True값이 많아질 것이고 재현율 값이 올라가고 정밀도는 떨어질 것이다.

임계값을 0.4에서부터 0.6까지 0.05씩 증가시키며 평가 지표를 조사해보자, 이를 위해 get_eval_by_threshold() 함수를 만듦

```python
# 테스트를 수행할 모든 임계값을 리스트 객체로 저장
thresholds = [0.4,0.45,0.5,0.55,0.6]

def get_eval_by_threshold(y_test,pred_proba_c1,thresholds):
    # thresholds list 객체 내의 값을 차례로 iteration하면서 Evaluation 수행.
    for custom_threshold in thresholds:
        binarizer = Binarizer(threshold=custom_threshold).fit(pred_proba_c1)
        custom_threshold = binarizer.transform(pres_proba_c1)
        print('임계값:',custom_threshold)
        get_clf_eval(y_test,custom_predict)
get_eval_by_threshold(y_test,pred_proba[:,1].reshapep(-1,1), thresholds)
```

- 임계값의 변경은 업무 환경에 맞게 두 개의 수치를 상호 보완할 수 있는 수준에서 적용돼야 한다. 